### Part 3.3 - Model Comparisons (Metrics & Plots)

The purpose of the notebook is to select the best model among the 5 models trained. 1 custom neural network and 4 neural network trained on pre-trained word-embeddings.
The final model selected will be then to proceed on the next part, the creation of the interactive conversational agent.

To select the best model we used the following guidelines:

* 1) The model with the lowest hamming loss & zero one loss.
* 2) The model with the lowest test score and the highest test accuracy values.
* 3) Compare each model's confusion matrix.
* 4) Training-Validation learning curves.
* 5) Bias-Variance trade-off.

Set the version data control parameter (to save the outputs of this notebook at their latest date)

In [ ]:
version_data_control="31072020"

#### Import the libraries

In [ ]:
import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections

import pandas as pd
import numpy as np
import os
import time
import random
import pickle
import json
import shutil
import unidecode

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from datetime import datetime
from tabulate import tabulate
from packaging import version
from humanfriendly import format_timespan
from sklearn.metrics import confusion_matrix, classification_report, hamming_loss, zero_one_loss, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
date_format='%Y-%m-%d %H-%M-%S'

#### Improt visualization libraries

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go

import pydot
import pydotplus
import graphviz

from IPython.display import SVG
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=False)
from pylab import rcParams

#### Import Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_docs as tfdocs #!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.plots as tfplots
import tensorflow_docs.modeling as tfmodel

from tensorflow.keras import layers, regularizers, models
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.models import load_model, model_from_json

#### Import Tensorflow Hub

In [ ]:
import tensorflow_hub as hub
from tensorboard.plugins.hparams import api as hp

#### Import Keras

In [ ]:
import keras.backend as K
from tensorflow import keras

In [ ]:
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

#### Import the data already tokenized and transformed from Part 3.1

* 80-20 split - Non-balanced data

In [ ]:
saved_version_data_control="13072020"
tokenization_history_folder="text_tokenization_padded_sequences"

with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\words_tokenized_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control)), 'rb') as handle:
    words_tokenized = pickle.load(handle)
words_tokenized

In [ ]:
"""
IMport the tokenizers of each input, fitted on part 3.1
"""
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\actors_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['actors_tokenized'])),'rb') as f:
    actors_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\plot_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['plot_words_tokenized'])),'rb') as f:
    plot_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\features_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['features_words_tokenized'])),'rb') as f:
    features_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\reviews_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['reviews_words_tokenized'])),'rb') as f:
    reviews_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\{0}_{1}\\title_tokenizer_{2}_{1}.pkl'.format(tokenization_history_folder, saved_version_data_control, words_tokenized['title_words_tokenized'])),'rb') as f:
    title_tokenizer = pickle.load(f)

try:
    assert len(actors_tokenizer.word_index)==words_tokenized['actors_tokenized']
    assert len(plot_tokenizer.word_index)==words_tokenized['plot_words_tokenized']
    assert len(features_tokenizer.word_index)==words_tokenized['features_words_tokenized']
    assert len(reviews_tokenizer.word_index)==words_tokenized['reviews_words_tokenized']
    assert len(title_tokenizer.word_index)==words_tokenized['title_words_tokenized']
except AssertionError:
    print("ERROR: The vocabulary length for some of the tokenizers, is not equal to 20000. Please verify their lengths by running the following: len(actors_tokenizer.word_index)")

print("Tokenizers are loaded successfully!")

In [ ]:
saved_version_data_control="13072020"
tokenization_history_folder="text_tokenization_padded_sequences"

X_train_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_actors_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['actors_tokenized'])))
X_train_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_plot_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['plot_words_tokenized'])))
X_train_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_features_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['features_words_tokenized'])))
X_train_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_reviews_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['reviews_words_tokenized'])))
X_train_seq_title=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_train_seq_title_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['title_words_tokenized'])))

print("X_train data inputs have been loaded!\n")

X_test_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_actors_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['actors_tokenized'])))
X_test_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_plot_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['plot_words_tokenized'])))
X_test_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_features_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['features_words_tokenized'])))
X_test_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_reviews_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['reviews_words_tokenized'])))
X_test_seq_title=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\x_test_seq_title_80-20_non-balanced_{2}_{1}.npy".format(tokenization_history_folder, saved_version_data_control, words_tokenized['title_words_tokenized'])))

print("X_test data inputs have been loaded!\n")

y_train=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\y_train_80-20_non-balanced_{1}.npy".format(tokenization_history_folder, saved_version_data_control)))
y_test=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\{0}_{1}\\y_test_80-20_non-balanced_{1}.npy".format(tokenization_history_folder, saved_version_data_control)))

print("y_train & y_test have been loaded!\n")

In [ ]:
X_train_seq_title.shape

In [ ]:
y_test.shape

#### Create the data that have been fiited on the pre-trained saved neural networks

In [ ]:
"""
Import the X_train, X_test, y_train & y_test data pickled from dataset part 3.1
"""
saved_version_data_control="13072020"

X_train=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\X_train_all_inputs_{0}.pkl".format(saved_version_data_control)))
X_test=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\X_test_all_inputs_{0}.pkl".format(saved_version_data_control)))
y_train_tf_hub=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\y_train_all_inputs_{0}.pkl".format(saved_version_data_control)))
y_test_tf_hub=pd.read_pickle(os.path.join(os.getcwd(), "pickled_data_per_part\\y_test_all_inputs_{0}.pkl".format(saved_version_data_control)))

assert X_train.shape[0]==y_train.shape[0]
assert X_test.shape[0]==y_test.shape[0]

In [ ]:
# Import genres

with open(os.path.join(os.getcwd(), 'pickled_data_per_part\\genres_list_06032020.pkl'.format(tokenization_history_folder, version_data_control)),'rb') as f:
    genres_list = pickle.load(f)
genres_list

In [ ]:
# Τhe creation of the data below and their fit on the model estimator will yield an overfitted model estimator
# numpy nd.arrays

train_text_features = X_train['clean_combined_features'].tolist() #input 1
test_text_features = X_test['clean_combined_features'].tolist()

train_text_plot = X_train['clean_plot_summary'].tolist() #input 2
test_text_plot = X_test['clean_plot_summary'].tolist()

train_text_actors = X_train['clean_actors'].tolist() #input 3
test_text_actors = X_test['clean_actors'].tolist()

train_text_reviews = X_train['clean_reviews'].tolist() #input 4
test_text_reviews = X_test['clean_reviews'].tolist()

train_text_title = X_train['clean_movie_title'].tolist() #input 5
test_text_title = X_test['clean_movie_title'].tolist()

train_label = y_train_tf_hub.values
test_label = y_test_tf_hub.values

train_bytes_list_features = []
train_bytes_list_plot = []
train_bytes_list_actors = []
train_bytes_list_reviews = []
train_bytes_list_title = []

test_bytes_list_features = []
test_bytes_list_plot = []
test_bytes_list_actors = []
test_bytes_list_reviews = []
test_bytes_list_title = []

train_bytes_list_features=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_features])
train_bytes_list_plot=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_plot])
train_bytes_list_actors=np.array([list(map(lambda x: str.encode(unidecode.unidecode(x)), i.split(','))) for i in train_text_actors])
# train_bytes_list_actors=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_actors])
train_bytes_list_reviews=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_reviews])
train_bytes_list_title=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_title])

test_bytes_list_features=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_features])
test_bytes_list_plot=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_plot])
test_bytes_list_actors=np.array([list(map(lambda x: str.encode(unidecode.unidecode(x)), i.split(','))) for i in test_text_actors])
# test_bytes_list_actors=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_actors])
test_bytes_list_reviews=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_reviews])
test_bytes_list_title=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_title])

partial_x_train_features, x_val_features, partial_y_train, y_val = train_test_split(train_bytes_list_features, train_label, test_size=0.20, random_state=42)
partial_x_train_plot, x_val_plot, partial_y_train, y_val = train_test_split(train_bytes_list_plot, train_label, test_size=0.20, random_state=42)
partial_x_train_actors, x_val_actors, partial_y_train, y_val = train_test_split(train_bytes_list_actors, train_label, test_size=0.20, random_state=42)
partial_x_train_reviews, x_val_reviews, partial_y_train, y_val = train_test_split(train_bytes_list_reviews, train_label, test_size=0.20, random_state=42)
partial_x_train_title, x_val_title, partial_y_train, y_val = train_test_split(train_bytes_list_title, train_label, test_size=0.20, random_state=42)

assert partial_x_train_actors.shape[0]==partial_x_train_plot.shape[0]==partial_x_train_features.shape[0]==partial_x_train_reviews.shape[0]==partial_x_train_title.shape[0]==partial_y_train.shape[0]

In [ ]:
# Option 1: Fit the 5 inputs as expected by the TF Hub Model.

partial_x_train_features_option1=partial_x_train_features.astype(np.object)
partial_x_train_plot_option1=partial_x_train_plot.astype(np.object)
partial_x_train_reviews_option1=partial_x_train_reviews.astype(np.object)
partial_x_train_title_option1=partial_x_train_title.astype(np.object)
partial_x_train_actors_option1=np.asarray([b" ".join(i) for i in partial_x_train_actors]).astype(np.object)

test_bytes_list_features_option1=test_bytes_list_features.astype(np.object)
test_bytes_list_plot_option1=test_bytes_list_plot.astype(np.object)
test_bytes_list_reviews_option1=test_bytes_list_reviews.astype(np.object)
test_bytes_list_title_option1=test_bytes_list_title.astype(np.object)
test_bytes_list_actors_option1=np.asarray([b" ".join(i) for i in test_bytes_list_actors]).astype(np.object)

x_val_features_option1=partial_x_train_features.astype(np.object)
x_val_plot_option1=x_val_plot.astype(np.object)
x_val_reviews_option1=x_val_reviews.astype(np.object)
x_val_title_title_option1=x_val_title.astype(np.object)
x_val_actors_option1=np.asarray([b" ".join(i) for i in x_val_actors]).astype(np.object)

#### Define the functions to be used across the notebook

In [ ]:
neural_network_parameters={}
optimizer_parameters={}

#----------------------------------------------------------------------

# Model Compilation
neural_network_parameters['model_loss'] = tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy') #'binary_crossentropy'
neural_network_parameters['model_metric'] = [tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
                                             tfa.metrics.F1Score(y_train.shape[-1], average="micro", name="f1_score_micro"), 
                                             tfa.metrics.F1Score(y_train.shape[-1], average=None, name="f1_score_none"),
                                             tfa.metrics.F1Score(y_train.shape[-1], average="macro", name="f1_score_macro")]

#----------------------------------------------------------------------
# Function 1 - Optimizers

# Optimizer: ADAM (Learning scheduler with Inverse Time Decay)

optimizer_parameters['lr_scheduler_decay_rate'] = 0.1
optimizer_parameters['staircase'] = False
optimizer_parameters['validation_split_ratio']=0.7

def optimizer_adam_v2(hparams):

    return keras.optimizers.Adam(tf.keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate=hparams[HP_LEARNING_RATE],
        decay_steps=int(np.ceil((X_train.shape[0]*optimizer_parameters['validation_split_ratio'])//hparams[HP_HIDDEN_UNITS]))*hparams[HP_DECAY_STEPS_MULTIPLIER],
        decay_rate=optimizer_parameters['lr_scheduler_decay_rate'],
        staircase=optimizer_parameters['staircase']))

#----------------------------------------------------------------------

# Function 2

def hamming_loss(y_true, y_pred, mode='multilabel'):
    if mode not in ['multiclass', 'multilabel']:
        raise TypeError('mode must be: [multiclass, multilabel]')

    if mode == 'multiclass':
        nonzero = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=-1), tf.float32)
        return 1.0 - nonzero

    else:
        nonzero = tf.cast(tf.math.count_nonzero(y_true - y_pred, axis=-1), tf.float32)
        return nonzero / y_true.shape[-1]

class HammingLoss(tfa.metrics.MeanMetricWrapper):
    def __init__(self, name='hamming_loss', dtype=None, mode='multilabel'):
        super(HammingLoss, self).__init__(
                hamming_loss, name, dtype=dtype, mode=mode)
        
#----------------------------------------------------------------------

# Function 3

def import_trained_keras_model(method, decay_steps_mode, embedding_dim_mode, optimizer_name, hparams):
    """
    Load the weights of the model saved with EarlyStopping
    """
    if method == "import custom trained model":
        
        if decay_steps_mode=="on":
            
            if embedding_dim_mode=="on":
            
                with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.json".format(folder_path_model_saved,
                                                                                                                            saved_model_name,
                                                                                                                            str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                            str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                            str(hparams[HP_LEARNING_RATE]), 
                                                                                                                            str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                            saved_model_version)),'r') as f:
                    model_json = json.load(f)

                model_imported = model_from_json(model_json)

                model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.h5'.format(folder_path_model_saved,
                                                                                                                                            saved_model_name,
                                                                                                                                            str(hparams[HP_EMBEDDING_DIM]),
                                                                                                                                            str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                            str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                            str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                            saved_model_version)))
            else:
                
                with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.json".format(folder_path_model_saved,
                                                                                                                     saved_model_name,
                                                                                                                     str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                     str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                     saved_model_version)),'r') as f:
                    model_json = json.load(f)

                model_imported = model_from_json(model_json)

                model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.h5'.format(folder_path_model_saved,
                                                                                                                                            saved_model_name,
                                                                                                                                            str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                                            str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                            str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                            saved_model_version)))
        else:
            
            with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.json".format(folder_path_model_saved,
                                                                                                     saved_model_name,
                                                                                                     str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                     str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                     saved_model_version)),'r') as f:
                model_json = json.load(f)

            model_imported = model_from_json(model_json)

            model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.h5'.format(folder_path_model_saved,
                                                                                                                     saved_model_name,
                                                                                                                     str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                     str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                     saved_model_version))) 
        if optimizer_name=="adam":
            optimizer = optimizer_adam_v2(hparams)
        
        elif optimizer_name=="sgd":
            optimizer = optimizer_sgd_v1(hparams, "step decay")
            
        else:
            optimizer = optimizer_rmsprop_v1(hparams)
            
        model_imported.compile(optimizer=optimizer,
                               loss=neural_network_parameters['model_loss'],
                               metrics=neural_network_parameters['model_metric'])
        print("\nModel is loaded successfully\n")
    
    else:
        
        with open(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.json".format(folder_path_model_saved,
                                                                                                             saved_model_name,
                                                                                                             str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                             str(hparams[HP_LEARNING_RATE]), 
                                                                                                             str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                             saved_model_version)),'r') as f:
            model_json = json.load(f)

        model_imported = model_from_json(model_json, custom_objects={'KerasLayer':hub.KerasLayer})

        model_imported.load_weights(os.path.join(os.getcwd(), '{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.h5'.format(folder_path_model_saved,
                                                                                                                             saved_model_name,
                                                                                                                             str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                             str(hparams[HP_LEARNING_RATE]), 
                                                                                                                             str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                             saved_model_version)))

        optimizer = optimizer_adam_v2(hparams)

        model_imported.compile(optimizer=optimizer,
                               loss=neural_network_parameters['model_loss'],
                               metrics=neural_network_parameters['model_metric'])
        print("\nModel is loaded successfully\n")
    
    return model_imported

#----------------------------------------------------------------------

# Function 2

def create_df_scoring_table(method, decay_steps_mode, embedding_dim_mode, model_tag, hparams, model):
    """
    Create a scoring dictionary to select the best out of the four models
    """
    if method == "import custom trained model":
        
        model_evaluation = model.evaluate([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title],
                                          y_test,
                                          batch_size=hparams[HP_HIDDEN_UNITS],
                                          verbose=2)

        y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        variance = np.var(y_test_predictions)
        sse = np.mean((np.mean(y_test_predictions) - y_test)**2)
        bias = sse - variance

        hamming_loss_value = HammingLoss(mode='multilabel')
        hamming_loss_value.update_state(y_test, y_test_predictions)
        
        if decay_steps_mode=="on":
            
            if embedding_dim_mode=="on":
            
                df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                        'Embedding Dimension tag':pd.Series(hparams[HP_EMBEDDING_DIM], dtype='int'),
                                        'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                        'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                        'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                        'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                        'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                        'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                        'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                        'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                        'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                        'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                        'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                        'Bias':pd.Series([bias], dtype='float'),
                                        'Variance':pd.Series([variance], dtype='float')
                                       })

                df_scores.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                                     saved_df_scored_metric_name,
                                                                                                                                     str(hparams[HP_EMBEDDING_DIM]),
                                                                                                                                     str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                                     str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                     saved_model_version)))
            else:
                
                df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                        'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                        'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                        'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                        'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                        'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                        'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                        'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                        'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                        'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                        'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                        'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                        'Bias':pd.Series([bias], dtype='float'),
                                        'Variance':pd.Series([variance], dtype='float')
                                       })

                df_scores.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                              saved_df_scored_metric_name,
                                                                                                                              str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                              str(hparams[HP_LEARNING_RATE]),
                                                                                                                              str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                              saved_model_version)))    
        else:
            
            df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                    'Embedding Dimension tag':pd.Series(hparams[HP_EMBEDDING_DIM], dtype='int'),
                                    'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                    'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                    'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                    'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                    'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                    'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                    'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                    'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                    'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                    'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                    'Bias':pd.Series([bias], dtype='float'),
                                    'Variance':pd.Series([variance], dtype='float')
                                   })

            df_scores.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(folder_path_model_saved,
                                                                                                              saved_df_scored_metric_name,
                                                                                                              str(hparams[HP_EMBEDDING_DIM]),
                                                                                                              str(hparams[HP_HIDDEN_UNITS]),
                                                                                                              str(hparams[HP_LEARNING_RATE]),
                                                                                                              saved_model_version)))
    else:
        
        model_evaluation = model.evaluate([test_bytes_list_plot_option1, test_bytes_list_features_option1, test_bytes_list_reviews_option1, test_bytes_list_title_option1],
                                          test_label,
                                          batch_size=hparams[HP_HIDDEN_UNITS],
                                          verbose=2)

        y_test_pred_probs = model.predict([test_bytes_list_plot_option1, test_bytes_list_features_option1, test_bytes_list_reviews_option1, test_bytes_list_title_option1])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        variance = np.var(y_test_predictions)
        sse = np.mean((np.mean(y_test_predictions) - test_label)**2)
        bias = sse - variance

        hamming_loss_value = HammingLoss(mode='multilabel')
        hamming_loss_value.update_state(test_label, y_test_predictions)

        df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                'Zero_one Loss':pd.Series([zero_one_loss(test_label, y_test_predictions, normalize=False)], dtype='float'),
                                'F1_score':pd.Series([f1_score(test_label, y_test_predictions, average="micro")], dtype='float'),
                                'F1_score_samples':pd.Series([f1_score(test_label, y_test_predictions, average="samples")], dtype='float'),
                                'ROC_score':pd.Series([roc_auc_score(test_label, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                'ROC_score_samples':pd.Series([roc_auc_score(test_label, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                'Bias':pd.Series([bias], dtype='float'),
                                'Variance':pd.Series([variance], dtype='float')
                               })

        df_scores.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                      saved_df_scored_metric_name,
                                                                                                                      str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                      str(hparams[HP_LEARNING_RATE]), 
                                                                                                                      str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                      saved_model_version)))
        
    
    return df_scores

#----------------------------------------------------------------------

# Function 5

def create_confusion_matrix(mode, decay_steps_mode, embedding_dim_mode,  model, hparams):

    if mode == "custom trained model":
        
        if decay_steps_mode=="on":
            
            if embedding_dim_mode=="on":
        
                y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
                y_test_predictions = (y_test_pred_probs>0.5).astype(int)

                conf_mat=confusion_matrix(y_test.argmax(axis=1), y_test_predictions.argmax(axis=1))

                conf_matrix=pd.DataFrame(conf_mat,
                                         columns=genres_list,
                                         index=genres_list)

                conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                                       "confusion_matrix",
                                                                                                                                       str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                                       str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                       str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                       str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                       saved_model_version)))
            else:
                
                y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
                y_test_predictions = (y_test_pred_probs>0.5).astype(int)

                conf_mat=confusion_matrix(y_test.argmax(axis=1), y_test_predictions.argmax(axis=1))

                conf_matrix=pd.DataFrame(conf_mat,
                                         columns=genres_list,
                                         index=genres_list)

                conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                                "confusion_matrix",
                                                                                                                                str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                                str(hparams[HP_LEARNING_RATE]),
                                                                                                                                str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                saved_model_version)))
        else:
            y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
            y_test_predictions = (y_test_pred_probs>0.5).astype(int)

            conf_mat=confusion_matrix(y_test.argmax(axis=1), y_test_predictions.argmax(axis=1))

            conf_matrix=pd.DataFrame(conf_mat,
                                     columns=genres_list,
                                     index=genres_list)

            conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(folder_path_model_saved,
                                                                                                                "confusion_matrix",
                                                                                                                str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                str(hparams[HP_LEARNING_RATE]),
                                                                                                                saved_model_version)))
    else:
        
        y_test_pred_probs = model.predict([test_bytes_list_plot_option1, test_bytes_list_features_option1, test_bytes_list_reviews_option1, test_bytes_list_title_option1])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        conf_mat=confusion_matrix(test_label.argmax(axis=1), y_test_predictions.argmax(axis=1))

        conf_matrix=pd.DataFrame(conf_mat,
                                 columns=genres_list,
                                 index=genres_list)

        conf_matrix.to_pickle(os.path.join(os.getcwd(), "{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                        "confusion_matrix",
                                                                                                                        str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                        str(hparams[HP_LEARNING_RATE]), 
                                                                                                                        str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                        saved_model_version)))
    return conf_matrix

#### Model 1 - Multi-Input Keras neural network (32 dim - 100 hidden units per hidden layer - 0.001 lr - 10 decay steps multiplier)

In [ ]:
# Custom Neural Network

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32]))
HP_EMBEDDING_DIM = hp.HParam('embedding_dim', hp.Discrete([100]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for embedding_dim in HP_EMBEDDING_DIM.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
                hparams_adam = {
                    HP_HIDDEN_UNITS: batch_size,
                    HP_EMBEDDING_DIM: embedding_dim,
                    HP_LEARNING_RATE: learning_rate,
                    HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
                  }
                saved_model_version="16072020"
                folder_path_model_saved="model_one\\adam_v2_models_20072020"
                saved_model_name="multi_input_keras_model"
                saved_df_scored_metric_name="df_metrics_multi_input_keras_model"
                model_one = import_trained_keras_model("import custom trained model", "on", "on", "adam", hparams_adam)
                df_one=pd.read_pickle("{0}\\{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved, saved_df_scored_metric_name, embedding_dim, batch_size, learning_rate, decay_steps_multiplier, saved_model_version))
                confusion_matrix_one=create_confusion_matrix("custom trained model", "on", "on", model_one, hparams_adam)
                history_dataframe_one=pd.read_pickle(os.path.join(os.getcwd(), "{0}\\metrics_histogram_multi_input_keras_{1}dim_{2}batchsize_{3}lr_{4}decaymultiplier_16072020.pkl".format(folder_path_model_saved, str(embedding_dim), str(batch_size), str(learning_rate), str(decay_steps_multiplier))))

#### Model 2 - Neural network trained with GloVe embeddings (64 hidden units per hidden layer - 0.001lr - 20 decay step multiplier)

In [ ]:
# GloVe embeddings

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([64]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([20]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for learning_rate in HP_LEARNING_RATE.domain.values:
        for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
            hparams_glove = {
                HP_HIDDEN_UNITS: batch_size,
                HP_LEARNING_RATE: learning_rate,
                HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
              }
            saved_model_version="18072020"
            folder_path_model_saved="model_two\\glove_models_20072020"
            saved_model_name="glove_embeddings"
            saved_df_scored_metric_name="df_metrics_glove"
            model_two = import_trained_keras_model("import custom trained model", "on", "off", "adam", hparams_glove)
            df_two=pd.read_pickle("{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved, saved_df_scored_metric_name, batch_size, learning_rate, decay_steps_multiplier, saved_model_version))
            confusion_matrix_two=create_confusion_matrix("custom trained model", "on", "off", model_two, hparams_glove)
            history_dataframe_two=pd.read_pickle(os.path.join(os.getcwd(), "{0}\\metrics_histogram_glove_embeddings_{1}batchsize_{2}lr_{3}decaymultiplier_18072020.pkl".format(folder_path_model_saved, str(batch_size), str(learning_rate), str(decay_steps_multiplier))))            

#### Model 3 - English Google News 130gb corpus without OOV token (32 hidden units per hidden layer - 0.01lr - 10 decay step multiplier)

In [ ]:
# English_Google_News_130GB_20dim_without_OOV

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.01]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for learning_rate in HP_LEARNING_RATE.domain.values:
        for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
            hparams_three = {
                HP_HIDDEN_UNITS: batch_size,
                HP_LEARNING_RATE: learning_rate,
                HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
              }
            saved_model_version="22072020"
            folder_path_model_saved="model_three\\english_google_news_20dim_no_OOV"
            saved_model_name="English_Google_News_130GB_20dim_without_OOV"
            saved_df_scored_metric_name="df_metrics_English_Google_News_130GB_20dim_without_OOV"
            model_three = import_trained_keras_model("import pre-trained model", "on", "off", "adam", hparams_three)
            df_three=pd.read_pickle("{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved, saved_df_scored_metric_name, batch_size, learning_rate, decay_steps_multiplier, saved_model_version))
            confusion_matrix_three=create_confusion_matrix("custom pre-trained model", "on", "off", model_three, hparams_three)
            history_dataframe_three=pd.read_pickle(os.path.join(os.getcwd(), "{0}\\metrics_histogram_English_Google_News_130GB_20dim_without_OOV_{1}batchsize_{2}lr_{3}decaymultiplier_22072020.pkl".format(folder_path_model_saved, str(batch_size), str(learning_rate), str(decay_steps_multiplier))))                    

#### Model 4 - English Google News 130gb corpus with OOV token (16 hidden units per hidden layer - 0.001lr - 20 decay steps multiplier)

In [ ]:
# English_google_news_130GB_20dim_with_oov

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([16]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([20]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for learning_rate in HP_LEARNING_RATE.domain.values:
        for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
            hparams_four = {
                HP_HIDDEN_UNITS: batch_size,
                HP_LEARNING_RATE: learning_rate,
                HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
              }
            saved_model_version="22072020"
            folder_path_model_saved="model_four\\english_google_news_20dim_with_OOV"
            saved_model_name="model_english_google_news_130GB_20dim_with_oov"
            saved_df_scored_metric_name="df_metrics_English_Google_News_130GB_20dim_with_OOV"
            model_four = import_trained_keras_model("import pre-trained model", "on", "off", "adam", hparams_four)
            df_four=pd.read_pickle("{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved, saved_df_scored_metric_name, batch_size, learning_rate, decay_steps_multiplier, saved_model_version))
            confusion_matrix_four=create_confusion_matrix("custom pre-trained model", "on", "off", model_four, hparams_four)
            history_dataframe_four=pd.read_pickle(os.path.join(os.getcwd(), "{0}\\metrics_histogram_english_google_news_130GB_20dim_with_oov_{1}batchsize_{2}lr_{3}decaymultiplier_22072020.pkl".format(folder_path_model_saved, str(batch_size), str(learning_rate), str(decay_steps_multiplier))))                                

#### Model 5 - English Google News 7B corpus with OOV token (32 hidden units per hidden layer - 0.01lr - 20 decay steps multiplier)

In [ ]:
# English_google_news_7B_50dim

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.01]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([20]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for learning_rate in HP_LEARNING_RATE.domain.values:
        for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
            hparams_five = {
                HP_HIDDEN_UNITS: batch_size,
                HP_LEARNING_RATE: learning_rate,
                HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
              }
            saved_model_version="22072020"
            folder_path_model_saved="model_five\\english_google_news_7B_50dim"
            saved_model_name="model_english_google_news_7B_50dim"
            saved_df_scored_metric_name="df_metrics_english_google_news_7B_50dim_with_OOV"
            model_five = import_trained_keras_model("import pre-trained model", "on", "off", "adam", hparams_five)
            df_five=pd.read_pickle("{0}\\{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved, saved_df_scored_metric_name, batch_size, learning_rate, decay_steps_multiplier, saved_model_version))
            confusion_matrix_five=create_confusion_matrix("custom pre-trained model", "on", "off", model_five, hparams_five)
            history_dataframe_five=pd.read_pickle(os.path.join(os.getcwd(), "{0}\\metrics_histogram_english_google_news_7B_50dim_{1}batchsize_{2}lr_{3}decaymultiplier_22072020.pkl".format(folder_path_model_saved, str(batch_size), str(learning_rate), str(decay_steps_multiplier))))                                            

# <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

**Comparison 1: Create a scoring dataframe for each model & comment on the resutls**

Step 1 of the selection plan (based on the written thesis documentation)

In [ ]:
# All frames together
frames_df = [df_one, df_two, df_three, df_four, df_five]
results_df= pd.concat(frames_df)
results_df=results_df.reset_index(drop=True)
results_df.sort_values(by=['Hamming Loss', 'Zero_one Loss'])

In [ ]:
print(results_df.to_latex(index=False))

**Comment:** From the correlation dataframe above, we spotted that Hamming Loss and Test Accuracy have a perfect negative correlation.
Thus, we can conclude that the model with the lowest Hamming Loss will also be the model with the highest Test Accuracy.
To validate this we checked the dataframe with the model results, and indeed model four what the highest accuracy on the test data.

**Comparison 2: Create the confusion matrix of each model**

In [ ]:
print(confusion_matrix_one.to_latex(index=True))

In [ ]:
print(confusion_matrix_two.to_latex(index=True))

In [ ]:
print(confusion_matrix_three.to_latex(index=True))

In [ ]:
print(confusion_matrix_four.to_latex(index=True))

In [ ]:
print(confusion_matrix_five.to_latex(index=True))

#### Comparison 3: Training and Validation learning curves

#### Performance Metric: Hamming Loss

In [ ]:
colormin = 'black'
colormax = 'black'
colorother = 'rgb(252, 141, 98)'

clrs_acc_model_one = [colormax if history_dataframe_one.val_hamming_loss.iloc[row]==history_dataframe_one.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_one.val_hamming_loss))]
clrs_acc_model_two = [colormax if history_dataframe_two.val_hamming_loss.iloc[row]==history_dataframe_two.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_two.val_hamming_loss))]
clrs_acc_model_three = [colormax if history_dataframe_three.val_hamming_loss.iloc[row]==history_dataframe_three.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_three.val_hamming_loss))]
clrs_acc_model_four = [colormax if history_dataframe_four.val_hamming_loss.iloc[row]==history_dataframe_four.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_four.val_hamming_loss))]
clrs_acc_model_five= [colormax if history_dataframe_five.val_hamming_loss.iloc[row]==history_dataframe_five.val_hamming_loss.min() else colorother for row in range(len(history_dataframe_five.val_hamming_loss))]

#Hamming Loss of the Custom trained Neural Netowrk
fig1=go.Figure()

fig1.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig1.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(), 
                          y=history_dataframe_one.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_one)))

fig1.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the Custom trained Neural Netowrk)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig1.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_one.epoch[history_dataframe_one.val_hamming_loss==history_dataframe_one.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_one.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation hamming loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig1.update_layout(legend_title_text='Training & Validation hamming loss points per epoch')

fig1.show()
#---------------------------------------------------------

#Hamming loss of the GloVe embeddings model
fig2=go.Figure()

fig2.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(), 
                          y=history_dataframe_two.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig2.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(), 
                          y=history_dataframe_two.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_two)))

fig2.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the GloVe embeddings Neural Network)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig2.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_two.epoch[history_dataframe_two.val_hamming_loss==history_dataframe_two.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_two.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation hamming loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig2.update_layout(legend_title_text='Training & Validation hamming loss points per epoch')

fig2.show()

#---------------------------------------------------------

#Hamming Loss of the pre-trained model English Google News 130GB 20dim without OOV token

fig3=go.Figure()

fig3.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(), 
                          y=history_dataframe_three.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig3.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(), 
                          y=history_dataframe_three.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_three)))

fig3.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the model English Google News 130GB 20dim without OOV token)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig3.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_three.epoch[history_dataframe_three.val_hamming_loss==history_dataframe_three.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_three.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation hamming loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig3.update_layout(legend_title_text='Training & Validation hamming loss points per epoch')

fig3.show()

#---------------------------------------------------------

#Hamming Loss of the pre-trained model English Google News 130GB 20dim with OOV token

fig4=go.Figure()

fig4.add_trace(go.Scatter(x=history_dataframe_four.epoch.tolist(), 
                          y=history_dataframe_four.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig4.add_trace(go.Scatter(x=history_dataframe_four.epoch.tolist(), 
                          y=history_dataframe_four.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_four)))

fig4.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the model English Google News 130GB 20dim with OOV token)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig4.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_four.epoch[history_dataframe_four.val_hamming_loss==history_dataframe_four.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_four.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation hamming loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig4.update_layout(legend_title_text='Training & Validation hamming loss points per epoch')

fig4.show()

#---------------------------------------------------------

#Hamming Loss of the pre-trained model English Google News 7B 50dim with OOV token

fig5=go.Figure()

fig5.add_trace(go.Scatter(x=history_dataframe_five.epoch.tolist(), 
                          y=history_dataframe_five.hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Training Hamming Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig5.add_trace(go.Scatter(x=history_dataframe_five.epoch.tolist(), 
                          y=history_dataframe_five.val_hamming_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Hamming Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_acc_model_five)))

fig5.update_layout(template="simple_white",
                   title="Hamming Loss score on train & validation sets (Model estimator of the model English Google News 7B 50dim with OOV token)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Hamming Loss/epoch")

fig5.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_five.epoch[history_dataframe_five.val_hamming_loss==history_dataframe_five.val_hamming_loss.min()].tolist()[0],
                                     y=history_dataframe_five.val_hamming_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation hamming loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=40)])

fig5.update_layout(legend_title_text='Training & Validation hamming loss points per epoch')

fig5.show()

#### Optimization Metric: Binary Cross Entropy or Sigmoid Focal Loss

In [ ]:
colormin = 'black'
colorother = 'rgb(252, 141, 98)'

clrs_loss_model_one=[colormin if history_dataframe_one.val_loss.iloc[row]==history_dataframe_one.val_loss.min() else colorother for row in range(len(history_dataframe_one.val_loss))]
clrs_loss_model_two=[colormin if history_dataframe_two.val_loss.iloc[row]==history_dataframe_two.val_loss.min() else colorother for row in range(len(history_dataframe_two.val_loss))]
clrs_loss_model_three=[colormin if history_dataframe_three.val_loss.iloc[row]==history_dataframe_three.val_loss.min() else colorother for row in range(len(history_dataframe_three.val_loss))]
clrs_loss_model_four=[colormin if history_dataframe_four.val_loss.iloc[row]==history_dataframe_four.val_loss.min() else colorother for row in range(len(history_dataframe_four.val_loss))]
clrs_loss_model_five=[colormin if history_dataframe_five.val_loss.iloc[row]==history_dataframe_five.val_loss.min() else colorother for row in range(len(history_dataframe_five.val_loss))]


# Loss of the Custom trained Neural Netowrk
fig6=go.Figure()

fig6.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(),
                          y=history_dataframe_one.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig6.add_trace(go.Scatter(x=history_dataframe_one.epoch.tolist(),
                          y=history_dataframe_one.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_one)))

fig6.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the Custom trained Neural Netowrk)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig6.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_one.epoch[history_dataframe_one.val_loss==history_dataframe_one.val_loss.min()].tolist()[0],
                                     y=history_dataframe_one.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-40)])
fig6.show()

#---------------------------------------------------------

#Loss of SGD optimizer model
fig7=go.Figure()

fig7.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(),
                          y=history_dataframe_two.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig7.add_trace(go.Scatter(x=history_dataframe_two.epoch.tolist(),
                          y=history_dataframe_two.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_two)))

fig7.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the GloVe embeddings Neural Network)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig7.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_two.epoch[history_dataframe_two.val_loss==history_dataframe_two.val_loss.min()].tolist()[0],
                                     y=history_dataframe_two.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-40)])
fig7.show()

#---------------------------------------------------------

#Loss of the model English Google News 130GB 20dim without OOV token
fig8=go.Figure()

fig8.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(),
                          y=history_dataframe_three.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig8.add_trace(go.Scatter(x=history_dataframe_three.epoch.tolist(),
                          y=history_dataframe_three.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_three)))

fig8.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the model English Google News 130GB 20dim without OOV token)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig8.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_three.epoch[history_dataframe_three.val_loss==history_dataframe_three.val_loss.min()].tolist()[0],
                                     y=history_dataframe_three.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-40)])
fig8.show()

#---------------------------------------------------------

#Loss of the model English Google News 130GB 20dim with OOV token
fig9=go.Figure()

fig9.add_trace(go.Scatter(x=history_dataframe_four.epoch.tolist(),
                          y=history_dataframe_four.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig9.add_trace(go.Scatter(x=history_dataframe_four.epoch.tolist(),
                          y=history_dataframe_four.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_four)))

fig9.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the model English Google News 130GB 20dim with OOV token)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig9.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_four.epoch[history_dataframe_four.val_loss==history_dataframe_four.val_loss.min()].tolist()[0],
                                     y=history_dataframe_four.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-40)])
fig9.show()

#---------------------------------------------------------

#Loss of the pre-trained model English Google News 7B 50dim with OOV token
fig10=go.Figure()

fig10.add_trace(go.Scatter(x=history_dataframe_five.epoch.tolist(),
                          y=history_dataframe_five.loss.tolist(),
                          mode='lines+markers',
                          name='Training Loss',
                          line=dict(color='rgb(102, 194, 165)')))

fig10.add_trace(go.Scatter(x=history_dataframe_five.epoch.tolist(),
                          y=history_dataframe_five.val_loss.tolist(),
                          mode='lines+markers',
                          name='Validation Loss',
                          line=dict(color='rgb(252, 141, 98)'),
                          marker=dict(color=clrs_loss_model_five)))

fig10.update_layout(template="simple_white",
                   title="Loss score on train & validation sets (Model estimator of the pre-trained model English Google News 7B 50dim with OOV token)",
                   xaxis_title="Number of epochs",
                   yaxis_title="Loss/epoch")

fig10.update_layout(showlegend=True,
                   annotations=[dict(x=history_dataframe_five.epoch[history_dataframe_five.val_loss==history_dataframe_five.val_loss.min()].tolist()[0],
                                     y=history_dataframe_five.val_loss.min(),
                                     xref="x",yref="y",
                                     text="Epoch with the lowest validation loss",
                                     showarrow=True,
                                     arrowhead=5,
                                     ax=0,ay=-40)])
fig10.show()